# Finding base station neighbours via the Delaunay triangulation

We will reuse the work of Delphine PAQUIRY to find base stations' neighbours.

In [ ]:
# Importation of libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay


In [ ]:
# Importing database
df = pd.read_csv("../database/data.csv", sep=";", decimal=",")
df.head()

## Useful methods



### Creation of a graph based on delaunay triangulation

In [ ]:
import networkx as nx
from itertools import combinations

def delaunay_to_graph(delaunay_triangulation):
    """ Returns a Networkx Graph based on the Delaunay triangulation and the position of each node.
        
        Parameters
        ----------
        delaunay_triangulation : Delaunay
            Result of the Delaunay triangulation.

        Returns
        -------
        G : Graph
            A Networkx Graph graph.
        pos : dict
            The position of G's nodes.
    """
    G=nx.Graph()
    nodes = range(len(delaunay_triangulation.points))
    G.add_nodes_from(nodes) # adds nodes names (0 to number_of_points-1)

    for simplex in delaunay_triangulation.simplices:
        G.add_edges_from(combinations(simplex, 2))

    pos = dict(zip(nodes,delaunay_triangulation.points)) # gives each node his own position

    return G, pos

### Computation of the distance in the in km between 2 points

In [ ]:
import math

def distance_geographique(pt1, pt2):
    """ Computes the distance in km between the points pt1 and pt2 (coordinates longitude, latitude).
        
        Parameters
        ----------
        pt1, pt2 : tuple
            Coordinates of the points.

        Returns
        -------
        distance : float
            Distance between pt1 and pt2.
    """
    R = 6371  # average Earth radius in kilometers
    dlat = math.radians(pt2[1] - pt1[1])
    dlon = math.radians(pt2[0] - pt1[0])
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(pt1[1])) * math.cos(math.radians(pt2[1])) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = R * c
    
    return distance

### Quadrants

In [ ]:
def create_6_quadrants(ref_point, pos):
    angles = np.degrees(np.arctan2([pos[k][1] for k in pos] - pos[ref_point][1], [pos[k][0] for k in pos] - pos[ref_point][0]))
    angles = (angles + 360) % 360

    quadrants = dict()
    for ind in range(0, 301, 60):
        quadrants[f"{ind}_{ind+60}"] = np.where((angles >= ind) & (angles < ind + 60))[0]

    return quadrants

In [ ]:
def nearestNeighbour_in_quadrant(ref_point, quadrant, pos):
    """Si pas de voisin, renvoie le point"""
    min = np.inf
    nearestNeighbour = ref_point
    
    for pt2 in quadrant:
        dist = distance_geographique(pos[ref_point], pos[pt2])
        if((dist > 0) and (dist < min)):
            min = dist
            nearestNeighbour = pt2

    return nearestNeighbour

## Criterias

### Distance criteria

In [ ]:
def distance_criteria(G, pos, max_distance=15):
    """ Removes all the edges of G wich are longer than max_distance.
        
        Parameters
        ----------
        G : Graph
            A Networkx Graph graph.
        pos : dict
            The position of G's nodes.
        max_distance : int (default=15)
            The maximum distance between two connected nodes (in km).
    """
    for edge in G.edges:
        if(distance_geographique(pos[edge[0]],pos[edge[1]]) > max_distance):
            G.remove_edges_from([edge])

### Quadrant criteria

In [ ]:
def quadrant_criteria(G, pos):
    """ Removes all the edges of G wich doesn't respect the quadrant criteria.
        
        Parameters
        ----------
        G : Graph
            A Networkx Graph graph.
        pos : dict
            The position of G's nodes.
    """
    for node in pos.keys():
        quadrants = create_6_quadrants(node, pos)
        NN = set()
        for quad in quadrants.values():
            NN.add(nearestNeighbour_in_quadrant(node, quad, pos))
        for edge in G.edges:
            if((edge[0]==node) and (edge[1] not in NN)):
                G.remove_edges_from([edge])

## Plotting methods

### Plotting parameters

In [ ]:
def plot_params(title, ax, long_points, lat_points):
    """ Sets the right parameters for subplots axes.
        
        Parameters
        ----------
        title : string
            Title of the subplot.
        ax : Matplotlib Axes object (default=None)
            Draw the graph in the specified Matplotlib axes.
        long_points : array
            Longitude coordinates of the points.
        lat_points : array
            Latitude coordinates of the points.
    """
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    
    ax.set_title(title)

    ax.tick_params(
        reset=True,
        top=False,
        right=False
    )
    
    ax.set_xlim(min(long_points)-0.05, max(long_points)+0.05)
    ax.set_ylim(min(lat_points)-0.05, max(lat_points)+0.05)

### Plot of a delaunay triangulation

In [ ]:
def plot_delaunay(delaunay_triangulation, show=True, ax=None):
    """ Plots a Delaunay triangulation.
        
        Parameters
        ----------
        delaunay_triangulation : Delaunay
            Result of the Delaunay triangulation.
        show : bool (default=True)
            If True, shows the plot.
        ax : Matplotlib Axes object (default=None)
            Draw the graph in the specified Matplotlib axes.
    """
    if(ax!=None):
        ax.triplot(delaunay_triangulation.points[:,0], delaunay_triangulation.points[:,1], delaunay_triangulation.simplices, linewidth=1, c="lightblue")
        ax.plot(delaunay_triangulation.points[:,0], delaunay_triangulation.points[:,1], 'o', markersize=3, c="green")
        plot_params("Delaunay Triangulation", ax, delaunay_triangulation.points[:,0], delaunay_triangulation.points[:,1])
    else:
        plt.triplot(delaunay_triangulation.points[:,0], delaunay_triangulation.points[:,1], delaunay_triangulation.simplices, linewidth=1, c="lightblue")
        plt.plot(delaunay_triangulation.points[:,0], delaunay_triangulation.points[:,1], 'o', markersize=3, c="green")
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.tick_params(
            reset=True,
            top=False,
            right=False)
    if(show):
        plt.show()

### Plot of a Networkx Graph

In [ ]:
def plot_graph(G, pos, show=True, **kwargs):
    """ Plots a Networkx Graph.
        
        Parameters
        ----------
        G : Graph
            The Networkx Graph graph you want to plot.
        pos : dict
            The position of G's nodes.
        show : bool (default=True)
            If True, shows the plot.
        ax : Matplotlib Axes object (default=None)
            Draw the graph in the specified Matplotlib axes.
        title : string
            The title of the subplot (works with ax).
    """
    ax = kwargs.get('ax', None)
    title = kwargs.get('title', "Delaunay Graph")

    if(ax):
        nx.draw_networkx(G, pos, node_size=10, with_labels=False, node_color="green", edge_color="lightblue", ax=ax)
        plot_params(title, ax, [pos[k][0] for k in pos], [pos[k][1] for k in pos])
    else:
        nx.draw_networkx(G, pos, node_size=10, with_labels=False, node_color="green", edge_color="lightblue")
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.tick_params(
            reset=True,
            top=False,
            right=False)
    if(show):
        plt.show()

## First try on the Gard county

### Creation of the dataframe

In [ ]:
department = "Gard"

In [ ]:
df_gard = df.loc[df['nom_dep'] == department]
df_gard = df_gard.drop(columns=["code_op", "id_site_partage", "id_station_anfr", "nom_reg", "nom_dep", "insee_dep", "insee_com", "site_2g", "site_3g", "site_4g", "site_5g", "mes_4g_trim", "site_ZB", "site_DCC", "site_strategique", "site_capa_240mbps", "date_ouverturecommerciale_5g", "site_5g_700_m_hz", "site_5g_800_m_hz", "site_5g_1800_m_hz", "site_5g_2100_m_hz", "site_5g_3500_m_hz"])
df_gard.head()

### Base stations' number

In [ ]:
providers = list(np.unique(df_gard["nom_op"]))
nb_pro = len(providers)

df_proPerDep = pd.DataFrame({
    "nom_dep" : department,
    "Free Mobile"   : [None],
    "SFR"   : [None],
    "Orange"   : [None],
    "Bouygues Telecom"   : [None],
    "Total"   : [None]
})

for pro in providers: # number of sites per provider
    count = list(df_gard["nom_op"]).count(pro)
    df_proPerDep.loc[df_proPerDep["nom_dep"]==department, pro] = count
df_proPerDep.loc[df_proPerDep["nom_dep"]==department, "Total"] = len(df_gard["num_site"])

In [ ]:
df_proPerDep

### Delaunay triangulation on Free Mobile stations

In [ ]:
# Selecting the provider : Free Mobile
df_gard_free = df_gard.loc[df['nom_op'] == "Free Mobile"]
df_gard_free = df_gard_free.drop(columns=['nom_op'])
df_gard_free.head()

In [ ]:
# Creation of points couples for Delaunay
df_gard_free_points = np.array(df_gard_free[['longitude', 'latitude']])

In [ ]:
delaunay_gard_free = Delaunay(df_gard_free_points)

In [ ]:
plot_delaunay(delaunay_gard_free)

### Creating a graph based on delaunay triangulation

In [ ]:
G, pos = delaunay_to_graph(delaunay_gard_free)

In [ ]:
plot_graph(G,pos)

### Addition of criterias

In [ ]:
G_dist, pos_dist = delaunay_to_graph(delaunay_gard_free)
G_quad, pos_quad = delaunay_to_graph(delaunay_gard_free)

In [ ]:
distance_criteria(G_dist,pos_dist)

In [ ]:
quadrant_criteria(G_quad,pos_quad)

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(16,5))
# plot_delaunay(delaunay_gard_free,ax=axs[0],show=False)
plot_graph(G_quad,pos_quad,ax=axs[0],show=False, title='Quadrant criteria')
plot_graph(G_dist,pos_dist,ax=axs[1],show=False, title='Distance criteria')
fig.suptitle("Comparison of the Quadrant and Distance cirteria", fontsize=18, va='center')
plt.show()

### Comparison of the results

In [ ]:
distance_criteria(G,pos)

In [ ]:
quadrant_criteria(G,pos)

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(16,5))
plot_delaunay(delaunay_gard_free,ax=axs[0],show=False)
plot_graph(G,pos,ax=axs[1],show=False,title="Delaunay after reduction")
fig.suptitle("Comparison of the Delaunay Triangulation and its reduction of edges", fontsize=18, va='center')
plt.show()

## Big Delaunay on France
Only Free Mobile

In [ ]:
# Selecting the provider : Free Mobile
df_free = df.loc[df['nom_op'] == "Free Mobile"]
df_free = df_free.drop(columns=['nom_op'])
df_free.head()

In [ ]:
# Creation of points couples for Delaunay
df_free_points = np.array(df_free[['longitude', 'latitude']])

In [ ]:
delaunay_free = Delaunay(df_free_points)

In [ ]:
plot_delaunay(delaunay_free)

In [ ]:
G_france, pos_france = delaunay_to_graph(delaunay_free)

In [ ]:
distance_criteria(G_france,pos_france)

In [ ]:
quadrant_criteria(G_france,pos_france)

In [ ]:
plot_graph(G_france,pos_france)